In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import re
import requests
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV


In [2]:
with open('coffee_words.pickle','rb') as read_file:
    coffee = pickle.load(read_file)
with open('coffee_ratings.pickle','rb') as read_file:
    ratings = pickle.load(read_file)
with open('combined.pickle','rb') as read_file:
    combined = pickle.load(read_file)
with open('df.pickle','rb') as read_file:
    df = pickle.load(read_file)
with open('df_topic_breakdown.pickle','rb') as read_file:
    df_topic_breakdown = pickle.load(read_file)

In [3]:
df_topic_breakdown.head()

,roaster,origin,roast_level,rating,group,bright_floral_citrus,choc_woody_dark,tart_sweet_smooth,cacao_nut_clean,sweet_nut_pine,juicy_cacao_honey,red_berries,woody_nut_caramel,cherry_vinuous_choc
0,Jackrabbit Java,Costa Rica,Medium-Light,93,4,0.000000,0.045379,0.073064,0.000000,0.104257,0.000000,0.010795,0.000000,0.000000
1,Jackrabbit Java,"Nyamasheke District, Rwanda",Medium-Light,92,2,0.018180,0.000000,0.073098,0.063934,0.000000,0.009512,0.000000,0.006350,0.012827
2,Red Rooster Coffee Roaster,"Los Naranjos, La Argentina, Huila Department, ...",Light,96,2,0.017113,0.000000,0.077220,0.000000,0.027378,0.000000,0.007769,0.003145,0.022832
3,Paradise Roasters,"Huila, Colombia",Light,95,2,0.008159,0.000000,0.143630,0.006424,0.000000,0.000000,0.000000,0.000000,0.005296
4,Kakalove Cafe,"Antioquia Department, Colombia",Medium-Light,95,6,0.000000,0.037483,0.074243,0.009381,0.000000,0.000000,0.114548,0.000000,0.053789


In [28]:
df_topic_breakdown['origin_general']=df_topic_breakdown.origin.str.split(', ').str[-1]
df_topic_breakdown.head(2)

,roaster,origin,roast_level,rating,group,bright_floral_citrus,choc_woody_dark,tart_sweet_smooth,cacao_nut_clean,sweet_nut_pine,juicy_cacao_honey,red_berries,woody_nut_caramel,cherry_vinuous_choc,origin_general
0,Jackrabbit Java,Costa Rica,Medium-Light,93,4,0.00000,0.045379,0.073064,0.000000,0.104257,0.000000,0.010795,0.00000,0.000000,Costa Rica
1,Jackrabbit Java,"Nyamasheke District, Rwanda",Medium-Light,92,2,0.01818,0.000000,0.073098,0.063934,0.000000,0.009512,0.000000,0.00635,0.012827,Rwanda


In [29]:
topic_features = ['bright_floral_citrus', 'choc_woody_dark', 'tart_sweet_smooth','cacao_nut_clean', 'sweet_nut_pine', 'juicy_cacao_honey', 'red_berries','woody_nut_caramel', 'cherry_vinuous_choc']
df_topic_breakdown.groupby(by='origin_general')[topic_features].mean().sample(3)

,bright_floral_citrus,choc_woody_dark,tart_sweet_smooth,cacao_nut_clean,sweet_nut_pine,juicy_cacao_honey,red_berries,woody_nut_caramel,cherry_vinuous_choc
origin_general,,,,,,,,,
west-central Colombia.,0.034851,0.000000,0.014859,0.000000,0.000000,0.143576,0.006175,0.000000,0.000000
northern Guatemala,0.004527,0.015949,0.043942,0.014583,0.020230,0.001417,0.006353,0.030291,0.007101
Southern Mexico,0.004033,0.022787,0.006875,0.014646,0.002892,0.000000,0.000000,0.076174,0.010958
